In [ ]:
!pip install datasets

In [6]:
from datasets import load_from_disk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter
import re

In [7]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [8]:
# 데이터셋 불러오기
path = "../data/train_dataset/"
dataset = load_from_disk(path)
print(dataset)
print(dataset['train'])

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})


In [10]:
train_df = dataset_to_df(dataset["train"])

In [ ]:
import re
import pandas as pd

def find_parentheses_content(text):
    pattern = r'(.+?)(\([^\)]*[\u4e00-\u9fff가-힣a-zA-Z]+[^\)]*\))?$'
    match = re.match(pattern, text)
    if match and match.group(2):
        return match.group(2)
    return None

# answer_text 열에서 괄호 안의 내용 추출
train_df['parentheses_content'] = train_df['answer_text'].apply(find_parentheses_content)

# 괄호 내용이 있는 행만 필터링
df_with_parentheses = train_df[train_df['parentheses_content'].notna()]

# 결과 출력
print(f"총 {len(df_with_parentheses)} 개의 행에서 괄호 내용이 발견되었습니다.")
print("\n괄호 내용 예시:")
print(df_with_parentheses[['answer_text', 'parentheses_content']].head(10))

# 괄호 내용의 고유한 값과 그 개수 출력
print("\n고유한 괄호 내용과 그 개수:")
print(df_with_parentheses['parentheses_content'].value_counts().head(10))

In [ ]:
# None 값이 아닌 행만 필터링
df_with_parentheses = train_df[train_df['parentheses_content'].notna()]

# 결과 출력
if len(df_with_parentheses) > 0:
    print(f"총 {len(df_with_parentheses)} 개의 행에서 괄호 내용이 발견되었습니다.")
    print("\n괄호 내용 예시:")
    print(df_with_parentheses[['answer_text', 'parentheses_content']].head(10))

    # 괄호 내용의 고유한 값과 그 개수 출력
    print("\n고유한 괄호 내용과 그 개수:")
    print(df_with_parentheses['parentheses_content'].value_counts().head(10))
else:
    print("괄호 내용이 포함된 행이 없습니다.")

In [ ]:
import re

def extract_answer_and_parentheses(text):
    pattern = r'(.+?)(\([^\)]*[\u4e00-\u9fff가-힣a-zA-Z]+[^\)]*\))?$'
    match = re.match(pattern, text)
    if match:
        answer = match.group(1).strip()
        parentheses = match.group(2) if match.group(2) else None
        return answer, parentheses
    return text, None

# DataFrame에 적용
train_df['extracted_answer'] = ''
train_df['parentheses_content'] = ''

for idx, row in train_df.iterrows():
    answer, parentheses = extract_answer_and_parentheses(row['answer_text'])
    train_df.at[idx, 'extracted_answer'] = answer
    train_df.at[idx, 'parentheses_content'] = parentheses

# 결과 출력
print(train_df[['answer_text', 'extracted_answer', 'parentheses_content']].head(10))

# 괄호 내용이 있는 행만 필터링
df_with_parentheses = train_df[train_df['parentheses_content'].notna()]

print(f"\n총 {len(df_with_parentheses)} 개의 행에서 괄호 내용이 발견되었습니다.")
print("\n괄호 내용 예시:")
print(df_with_parentheses[['answer_text', 'extracted_answer', 'parentheses_content']].head(10))

In [16]:
df_with_parentheses.head()

,title,context,question,answer_start,answer_text,document_id,parentheses_content,extracted_answer
42,하프라이프의 등장인물 목록,그리고리 신부(Father Grigori)는 게임 하프라이프 2의 레이븐홈 챕터에서...,고든 프리맨이 레이븐홈을 떠날 수 있도록 도와준 사람은?,0,그리고리 신부(Father Grigori),24723,(Father Grigori),그리고리 신부
76,성당과 시장,에릭 레이먼드가 쓴 〈성당과 시장〉(The Cathedral and the Baza...,에릭 레이먼드의 자유 소프트웨어 철학을 대변하는 글은?,11,〈성당과 시장〉(The Cathedral and the Bazaar),4655,(The Cathedral and the Bazaar),〈성당과 시장〉
136,쾌락 원리의 저편,프로이트는 꿈을 해석해서 참다운 꿈 생각을 밝혀낼 수 있다고 믿었다. 또한 환자와 ...,프로이트의 정신 체계에서 현실적인 정신체계와 본능적 충동의 체계를 제외한 것의 명칭은?,202,‘전의식’(前意識),26108,(前意識),‘전의식’
137,도료,바니시 이외의 대표적인 천연도료로서 우리나라에서는 예부터 옻칠이 사용되었다. 옻칠은...,옻칠의 가장 중요한 요소는 무엇인가?,239,우루시올(urushiol),24413,(urushiol),우루시올
193,신촌에서 유턴하다,혜주(김여진 분)는 운전 면허시험에 번번이 낙방하고 연습 중에 급기야 사고까지 내면...,혜주의 운전교습을 잠시 맡게된 인물의 형은?,72,경석(김규철 분),52975,(김규철 분),경석


In [13]:
import re
import pandas as pd
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# parentheses_content가 있는 행만 필터링
df_with_parentheses = train_df[train_df['parentheses_content'].notna()]

# 언어 감지
df_with_parentheses['language'] = df_with_parentheses['parentheses_content'].apply(detect_language)

# 언어별 비율 계산
language_counts = df_with_parentheses['language'].value_counts()
language_percentages = language_counts / len(df_with_parentheses) * 100

print("언어별 개수:")
print(language_counts)

print("\n언어별 비율:")
print(language_percentages)

# 상위 5개 언어의 예시 출력
print("\n상위 5개 언어의 예시:")
for lang in language_counts.index[:5]:
    examples = df_with_parentheses[df_with_parentheses['language'] == lang]['parentheses_content'].head(3)
    print(f"\n{lang}:")
    for ex in examples:
        print(f"  {ex}")

언어별 개수:
language
ko       59
en       14
zh-cn     4
ca        3
de        2
ro        2
zh-tw     2
sw        1
tl        1
id        1
ja        1
cy        1
lt        1
Name: count, dtype: int64

언어별 비율:
language
ko       64.130435
en       15.217391
zh-cn     4.347826
ca        3.260870
de        2.173913
ro        2.173913
zh-tw     2.173913
sw        1.086957
tl        1.086957
id        1.086957
ja        1.086957
cy        1.086957
lt        1.086957
Name: count, dtype: float64

상위 5개 언어의 예시:

ko:
  (前意識)
  (김규철 분)
  (魯陵)

en:
  (Father Grigori)
  (The Cathedral and the Bazaar)
  (Oxford College of Emory University)

zh-cn:
  (鉱害)
  (媒槪念)
  (冬牲)

ca:
  (Militant)
  (Cataplexy)
  (Agave tequilana)

de:
  (Georg Bell)
  (GUI)


C:\Users\black\AppData\Local\Temp\ipykernel_27528\128930250.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_parentheses['language'] = df_with_parentheses['parentheses_content'].apply(detect_language)
